In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [19]:
import numpy as np 
import matplotlib.pyplot as plt 
import os
from PIL import Image
print(os.listdir("../input"))

# Keras Libraries
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, f1_score

['chest-xray-pneumonia']


In [32]:
mainDIR = os.listdir('../input/chest-xray-pneumonia/chest_xray')
print(mainDIR)

['chest_xray', '__MACOSX', 'val', 'test', 'train']


In [33]:
train_folder= '../input/chest-xray-pneumonia/chest_xray/train/' 
val_folder = '../input/chest-xray-pneumonia/chest_xray/val/'
test_folder = '../input/chest-xray-pneumonia/chest_xray/test/'

In [34]:
os.listdir(train_folder)
train_normal=train_folder+'NORMAL/'
train_pneumonia=train_folder+'PNEUMONIA/'

In [36]:
train_count = 0
for class_folder in os.listdir(train_folder):
    class_folder_path = os.path.join(train_folder, class_folder)
    if os.path.isdir(class_folder_path):  # Only count files in subdirectories
        train_count += len([f for f in os.listdir(class_folder_path) if os.path.isfile(os.path.join(class_folder_path, f))])

val_count = 0
for class_folder in os.listdir(val_folder):
    class_folder_path = os.path.join(val_folder, class_folder)
    if os.path.isdir(class_folder_path):  # Only count files in subdirectories
        val_count += len([f for f in os.listdir(class_folder_path) if os.path.isfile(os.path.join(class_folder_path, f))])

test_count = 0
for class_folder in os.listdir(test_folder):
    class_folder_path = os.path.join(test_folder, class_folder)
    if os.path.isdir(class_folder_path):  # Only count files in subdirectories
        test_count += len([f for f in os.listdir(class_folder_path) if os.path.isfile(os.path.join(class_folder_path, f))])

# Print the results
print(f"Number of images in train: {train_count}")
print(f"Number of images in val: {val_count}")
print(f"Number of images in test: {test_count}")

Number of images in train: 5216
Number of images in val: 16
Number of images in test: 624


In [47]:
cnn = Sequential()
cnn.add(Conv2D(32, (3, 3), activation="relu", input_shape=(64, 64, 3)))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Conv2D(32, (3, 3), activation="relu"))
cnn.add(MaxPooling2D(pool_size = (2, 2)))
cnn.add(Flatten())
cnn.add(Dense(activation = 'relu', units = 128))
cnn.add(Dense(activation = 'sigmoid', units = 1))
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy', 'AUC'])

In [48]:
trainData= ImageDataGenerator(rescale=1./255,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True
                             )

testData=ImageDataGenerator(rescale=1./255)

training_dataset=trainData.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/train',
                                              target_size=(64,64),
                                               batch_size=32,
                                               class_mode='binary'
                                              )

Validation= testData.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/val/',
                                              target_size=(64,64),
                                               batch_size=32,
                                               class_mode='binary'
                                              )

testing_dataset= trainData.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/test',
                                              target_size=(64,64),
                                               batch_size=32,
                                               class_mode='binary'
                                              )

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [49]:
cnn_model = cnn.fit(training_dataset,
                         steps_per_epoch = 163,
                         epochs = 1,
                         validation_data = Validation,
                         validation_steps = 624)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


163/163 ━━━━━━━━━━━━━━━━━━━━ 103s 587ms/step - AUC: 0.7235 - accuracy: 0.7662 - loss: 0.4898 - val_AUC: 0.9531 - val_accuracy: 0.7500 - val_loss: 0.5242


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [51]:
test_accu = cnn.evaluate(testing_dataset,steps=624)

624/624 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - AUC: 0.8980 - accuracy: 0.8017 - loss: 0.4769  


In [52]:
print('The testing accuracy is :',test_accu[1]*100, '%')

The testing accuracy is : 80.12820482254028 %


In [57]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess the image manually
img = image.load_img('../input/chest-xray-pneumonia/chest_xray/test/PNEUMONIA/person111_bacteria_534.jpeg', target_size=(64, 64))  
img_array = image.img_to_array(img)  # Convert the image to a NumPy array
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension


img_array = img_array / 255.0  

# prediction
prediction = cnn.predict(img_array)

# For binary classification (normal vs pneumonia)
predicted_class = (prediction > 0.5).astype(int)
if predicted_class[0] == 0:
    print("Normal")
else:
    print("Pneumonia")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Pneumonia
